# Build Map

## Map Show

In [181]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.melbourne.vic.gov.au", None)


CPBL = client.get("vh2v-4nfs", limit=2000)  
#On-street Parking Bay Sensors:
#https://data.melbourne.vic.gov.au/Transport/On-street-Parking-Bay-Sensors/vh2v-4nfs

CPBL_df = pd.DataFrame.from_records(CPBL)

CPBR = client.get("ntht-5rk7", limit=20000)
#On-street Car Park Bay Restrictions:
#https://data.melbourne.vic.gov.au/Transport/On-street-Car-Park-Bay-Restrictions/ntht-5rk7
CPBR_df = pd.DataFrame.from_records(CPBR)


#*****
merge_df=pd.merge(CPBL_df,CPBR_df,left_on='bay_id',right_on='bayid', how='left')

df=merge_df.iloc[:,[0,1,2,3,4,5,9,10,11,12,13,16,17,18]]
#df[['lat','lon','disabilityext1','duration1']].apply(pd.to_numeric)

#df['location']=eval(df['location'])
#from map import mark_map
#mark_map(df)

## Map Filter

In [182]:
from geopy.distance import geodesic
import numpy as np


def geofilter(pin,n):
    filter=[]
    for i in np.arange(0,df.shape[0]):
        d = geodesic(pin[0],df.iloc[i]['location']).km
        if d<=n:
            filter.iloc[i]= df.iloc[i]
        else:
            continue
    filter.to_csv('Result.csv')
    return filter,mark_map(filter)

In [183]:
d=dict(df['location'])
for i in np.arange(0,len(d)):
    del d[i]['human_address']
dictdf=pd.DataFrame([d])
df['location'] = pd.DataFrame(dictdf.values.T, index=dictdf.columns, columns=dictdf.index)

TypeError: 'dict' object is not callable

In [ ]:
#pin={'latitude': -37.82, 'longitude': 144.95}
pin=[(-37.82, 144.95)]
n=2
geofilter(pin,n)

ValueError: could not convert string to float: 'latitude'